## 3-step process to reconstruct paleolocations:
1. combine input points into feature collection
2. assign plate ids to features
3. reconstruct paleolocations for features


In [4]:
import sys
sys.path.insert(1, './pygplates_rev28_python38_MacOS64/')
import pygplates

import pandas as pd
import numpy as np

import os
import json
import netCDF4

# define plate model

# static polygons are the 'partitioning features'
static_polygons = pygplates.FeatureCollection('PALEOMAP_Global_Plate_Model/PALEOMAP_PlatePolygons.gpml')

# The partition_into_plates function requires a rotation model, since sometimes this would be
# necessary even at present day (for example to resolve topological polygons)
rotation_model=pygplates.RotationModel('PALEOMAP_Global_Plate_Model/PALEOMAP_PlateModel.rot')

In [12]:
# load list of simulations
ensemble       = 'scotese_04'

# Opening JSON file
cwd = os.getcwd()
f = open(cwd + '/climatearchive/' + ensemble + '.json')
 
# returns JSON object as
# a dictionary
ensembleList = json.load(f)
 
# Iterating through the json
# list
#for i in ensembleList['timeslice']:
#    print(i['tMin'])
    
    
# load HadCM3 example file to extract latitudes/longitudes
file2read = netCDF4.Dataset(cwd + '/climatearchive/teyea.qrparm.mask.shifted.nc','r')
# lat = file2read.variables['latitude'][:]  # access a variable in the file
# lon = file2read.variables['longitude'][:] # access a variable in the file
lon = np.arange(-180, 180.1, 2).tolist()
lat = np.arange(90, -90.1, -2).tolist()

# create array to store 2d fields for each timeslice
paleoLATS = np.zeros((len(ensembleList['timeslice']), len(lat), len(lon)))
paleoLONS = np.zeros((len(ensembleList['timeslice']), len(lat), len(lon)))
print(paleoLATS.shape)

lon[0] = -179.9
lon[-1] =  179.9
lat[0] = ( lat[0] + lat[1] ) / 2.
lat[-1] = lat[0] * -1.

print(lon)
print(lat)


(109, 91, 181)
[-179.9, -178.0, -176.0, -174.0, -172.0, -170.0, -168.0, -166.0, -164.0, -162.0, -160.0, -158.0, -156.0, -154.0, -152.0, -150.0, -148.0, -146.0, -144.0, -142.0, -140.0, -138.0, -136.0, -134.0, -132.0, -130.0, -128.0, -126.0, -124.0, -122.0, -120.0, -118.0, -116.0, -114.0, -112.0, -110.0, -108.0, -106.0, -104.0, -102.0, -100.0, -98.0, -96.0, -94.0, -92.0, -90.0, -88.0, -86.0, -84.0, -82.0, -80.0, -78.0, -76.0, -74.0, -72.0, -70.0, -68.0, -66.0, -64.0, -62.0, -60.0, -58.0, -56.0, -54.0, -52.0, -50.0, -48.0, -46.0, -44.0, -42.0, -40.0, -38.0, -36.0, -34.0, -32.0, -30.0, -28.0, -26.0, -24.0, -22.0, -20.0, -18.0, -16.0, -14.0, -12.0, -10.0, -8.0, -6.0, -4.0, -2.0, 0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0, 32.0, 34.0, 36.0, 38.0, 40.0, 42.0, 44.0, 46.0, 48.0, 50.0, 52.0, 54.0, 56.0, 58.0, 60.0, 62.0, 64.0, 66.0, 68.0, 70.0, 72.0, 74.0, 76.0, 78.0, 80.0, 82.0, 84.0, 86.0, 88.0, 90.0, 92.0, 94.0, 96.0, 98.0, 100.0, 102.0, 104.0, 1

In [15]:
valLats = [35,28,-20,-70,38,54,64,71,44,52,-50,78,-15,44]
valLons = [-70,-75,110,-13,-2,0,60,97,5,20,-72,16,117,-5]

point_features = []
for ii in range(len(valLats)):
    point = pygplates.PointOnSphere(float(valLats[ii]),float(valLons[ii]))
    point_feature = pygplates.Feature()
    point_feature.set_geometry(point)
    point_features.append(point_feature)

### step 2: assign plate ids to features
# To reconstruct any feature geometries, each feature must have a plate id assigned. If they don't already, 
# then the pygplates function 'PlatePartitioner' performs this function (analogous to the 'assign plate ids' 
# menu option in GPlates GUI) 
partitioned_point_features = pygplates.partition_into_plates(static_polygons, rotation_model, point_features) 

### step 3: reconstruct paleolocations for features 
#output paleolocations directly
reconstructed_feature_geometries = []
pygplates.reconstruct(partitioned_point_features, rotation_model, reconstructed_feature_geometries, 136.4 )  
#pygplates.reconstruct(partitioned_point_features, rotation_model, reconstructed_feature_geometries, 100.0)   

for lonCount, reconstructed_feature_geometry in enumerate(reconstructed_feature_geometries):
    pLocation = reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon()
    print('Paleolocation (LAT, LON) for LAT: ' + str(valLats[lonCount]) +':', reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon() )
    #paleoLATS[timeCount, jj, lonCount] = pLocation[0]
    #paleoLONS[timeCount, jj, lonCount] = pLocation[1]

Paleolocation (LAT, LON) for LAT: 35: (27.46156859289222, -23.163555811940007)
Paleolocation (LAT, LON) for LAT: 28: (21.130498321233745, -28.874074988087276)
Paleolocation (LAT, LON) for LAT: -20: (-45.21392741664934, 70.3419318886537)
Paleolocation (LAT, LON) for LAT: -70: (-48.65354338608867, -9.002537317994612)
Paleolocation (LAT, LON) for LAT: 38: (29.44263751408924, 6.227218703292507)
Paleolocation (LAT, LON) for LAT: 54: (44.768863494352736, 9.677845837259918)
Paleolocation (LAT, LON) for LAT: 64: (59.13539294445548, 56.014737623104054)
Paleolocation (LAT, LON) for LAT: 71: (70.91203380641811, 80.54592621379979)
Paleolocation (LAT, LON) for LAT: 44: (34.87641907613523, 14.347342186685918)
Paleolocation (LAT, LON) for LAT: 52: (43.51805374674896, 26.562965840708628)
Paleolocation (LAT, LON) for LAT: -50: (-49.13596274380457, -33.01303956950918)
Paleolocation (LAT, LON) for LAT: 78: (69.11697965906477, 17.31700087346007)
Paleolocation (LAT, LON) for LAT: -15: (-46.84739105744845, 

In [13]:
# create rotationTextures (paleolat and paleolon for modern lat/lon) for each time period

for timeCount,tt in enumerate(ensembleList['timeslice']):
       
    print(tt['tMin'])

    #### step 1: put all grid points into a feature collection 
    for jj in range(len(lat)):
        point_features = []
        for ii in range(len(lon)):
            point = pygplates.PointOnSphere(float(lat[jj]),float(lon[ii]))
            point_feature = pygplates.Feature()
            point_feature.set_geometry(point)
            point_features.append(point_feature)

        ### step 2: assign plate ids to features
        # To reconstruct any feature geometries, each feature must have a plate id assigned. If they don't already, 
        # then the pygplates function 'PlatePartitioner' performs this function (analogous to the 'assign plate ids' 
        # menu option in GPlates GUI) 
        partitioned_point_features = pygplates.partition_into_plates(static_polygons, rotation_model, point_features) 

        ### step 3: reconstruct paleolocations for features 
        #output paleolocations directly
        reconstructed_feature_geometries = []
        pygplates.reconstruct(partitioned_point_features, rotation_model, reconstructed_feature_geometries, float(tt['tMin'] * -1.))   
        #pygplates.reconstruct(partitioned_point_features, rotation_model, reconstructed_feature_geometries, 100.0)   

        for lonCount, reconstructed_feature_geometry in enumerate(reconstructed_feature_geometries):
            pLocation = reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon()
            #print('Paleolocation (LAT, LON) for LAT: ' + str(lat[jj]) + ' / LON: ' + str(lon[lonCount]) + ':', reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon() )
            paleoLATS[timeCount, jj, lonCount] = pLocation[0]
            paleoLONS[timeCount, jj, lonCount] = pLocation[1]
                

-541.0
-535.0
-530.0
-525.0
-520.0
-515.0
-510.0
-505.0
-498.8
-495.5
-491.8
-485.4
-481.6
-475.0
-470.0
-465.0
-460.0
-455.7
-449.1
-444.5
-441.2
-436.0
-430.4
-425.2
-421.1
-415.0
-409.2
-405.0
-400.0
-395.0
-390.5
-385.2
-380.0
-375.0
-370.0
-365.6
-358.9
-354.0
-349.0
-344.0
-338.8
-333.0
-330.9
-327.0
-319.2
-314.6
-311.1
-305.4
-301.3
-297.0
-292.6
-286.8
-280.0
-275.0
-268.7
-265.1
-262.5
-256.0
-252.0
-244.6
-239.5
-233.6
-232.0
-227.0
-222.4
-217.8
-213.2
-204.9
-201.3
-196.0
-190.8
-186.8
-178.4
-172.2
-168.2
-164.8
-160.4
-154.7
-148.6
-145.0
-142.4
-136.4
-131.2
-127.2
-121.8
-115.8
-111.0
-107.0
-102.6
-97.2
-91.9
-86.7
-80.8
-75.0
-69.0
-66.0
-61.0
-56.0
-51.9
-44.5
-39.5
-35.9
-31.0
-25.6
-19.5
-14.9
-10.5
-3.1
0.0


In [14]:
# write textures to disk

# function to normalize data to range 0-1
def norm(arr, norm_min, norm_max):
    arr_min =  norm_min
    arr_max =  norm_max
    return (arr - norm_min) / (norm_max - norm_min)
    

outputData  = np.zeros( ( len(lat), len(lon) * 109, 3) )

for tt in range(109):
    outputData[:,tt*len(lon) : ((tt+1)*len(lon)),0] = norm(paleoLATS[tt,:,:], -90., 90.) * 255 
    outputData[:,tt*len(lon) : ((tt+1)*len(lon)),1] = norm(paleoLONS[tt,:,:], -180., 180.) * 255 
#     outputData[:,tt*len(lon) : ((tt+1)*len(lon)),0] = norm(paleoLATS[tt,:,:], -90., 90.) * 65535 
#     outputData[:,tt*len(lon) : ((tt+1)*len(lon)),1] = norm(paleoLONS[tt,:,:], -180., 180.) * 65535 
# cap data to chosen dynamic range
outputData[outputData > 255] = 255
outputData[outputData < 0] = 0
outputData = outputData.astype(np.uint8)
# outputData[outputData > 65535] = 65535
# outputData[outputData < 0] = 0
# outputData = outputData.astype(np.uint16)
import png

with open(cwd + '/climatearchive/rotationTextures/' + ensemble + '.rotations.png', 'wb') as f:
    writer = png.Writer(width=outputData.shape[1], height=outputData.shape[0], bitdepth=8, greyscale=False)
#     writer = png.Writer(width=outputData.shape[1], height=outputData.shape[0], bitdepth=16, greyscale=False)
    # Convert outputData_3D_ym to the Python list of lists expected by the png writer.
    outputData_ym2list = outputData.reshape(-1, outputData.shape[1]*outputData.shape[2]).tolist()
    writer.write(f, outputData_ym2list)